# Item-item collaborative filtering

In [1]:
from pyspark.sql.functions import asc, first, mean, row_number, when, udf, stddev_pop, col, lit, collect_list, array
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.window import Window
from pyspark.ml.linalg import DenseVector
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import collect_list, udf, size
from pyspark.sql.types import ArrayType, DoubleType

## Overview
1. Load dataset
2. Split the dataset
3. Normalize the dataset playtime into a 1-5 rating scale
4. Compute Pearson correlation matrix
5. Rating prediction by doing a weighted average of the k most similar items that the user has played before

## Load Dataset
The dataset is loaded from MariaDB into a dataframe.

In [2]:
spark = SparkSession.builder.appName('ReadMariaDB') \
.config("spark.driver.memory", "32g") \
.config("spark.sql.pivotMaxValues", "1000000") \
.getOrCreate()

spark.sparkContext.setLogLevel("ERROR")


sql = "select * from 01_sampled_games_2v2 WHERE playtime_forever IS NOT NULL AND playtime_forever > 0"
database = "steam"
user = "root"
password = "example"
server = "192.168.2.62"
port = 3306
jdbc_url = f"jdbc:mysql://{server}:{port}/{database}?permitMysqlScheme"
jdbc_driver = "org.mariadb.jdbc.Driver"

# Create a data frame by reading data from Oracle via JDBC
df = spark.read.format("jdbc") \
    .option("url", jdbc_url) \
    .option("query", sql) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", jdbc_driver) \
    .load()

df = df.drop("playtime_2weeks", "dateretrieved")

23/04/04 15:57:37 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.122.1 instead (on interface virbr0)
23/04/04 15:57:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/04 15:57:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Count the number of rows in the DataFrame
row_count = df.count()

# Print the row count
print("The DataFrame has", row_count, "rows.")

df.show()

The DataFrame has 88349 rows.
+-----------------+-----+----------------+
|          steamid|appid|playtime_forever|
+-----------------+-----+----------------+
|76561197960268000|  300|             109|
|76561197960268000| 1300|              94|
|76561197960268000| 2100|             110|
|76561197960268000| 4000|             152|
|76561197960268000| 2600|              59|
|76561197960268000| 9000|               2|
|76561197960268000| 2300|              40|
|76561197960268000| 2200|             210|
|76561197960268000| 4500|            1002|
|76561197960268000|  400|             380|
|76561197960268000|17300|            4312|
|76561197960268000|  500|             198|
|76561197960268000|22300|             425|
|76561197960268000|19900|             111|
|76561197960268000|18500|            1160|
|76561197960268000|22200|              20|
|76561197960268000|35700|             433|
|76561197960268000|36000|              27|
|76561197960268000|39800|             510|
|76561197960268000|32400

## Split the dataset

In [4]:
# Randomly split the data into 70% training and 30% test data
training, test = df.randomSplit([0.7, 0.3], seed=1234)

## Per-Game 1-5 Rating Normalization
For each game, we calculate the mean and standard deviation. We then create buckets for each rating:
### Cut points
* Cut point 1: mean - std_dev*0.5 if > 0, else 0
* Cut point 2: mean
* Cut point 3: mean + std_dev*0.5
* Cut point 4: mean + std_dev
### Ratings
* Rating 1: 0 < x < cut point 1
* Rating 2: cut point 1 < x < cut point 2
* Rating 3: cut point 2 < x < cut point 3
* Rating 4: cut point 3 < x < cut point 4
* Rating 5: cut point 5 < x < inf

In [5]:
# Calculate the per-game mean and standard deviation of the playtime column
game_stats = training.filter(col("playtime_forever") > 0).groupBy("appid").agg(
    mean("playtime_forever").alias("game_mean_playtime"),
    stddev_pop("playtime_forever").alias("game_stddev_playtime")
)
training = training.join(game_stats, "appid")

training = training.withColumn("cut_point_1", when(col("game_mean_playtime") - (col("game_stddev_playtime") * 0.5) > 0, col("game_mean_playtime") - (col("game_stddev_playtime") * 0.5)).otherwise(0))
training = training.withColumn("cut_point_2", col("game_mean_playtime"))
training = training.withColumn("cut_point_3", col("game_mean_playtime") + (col("game_stddev_playtime") * 0.5))
training = training.withColumn("cut_point_4", col("game_mean_playtime") + col("game_stddev_playtime"))

training = training.withColumn(
    "ratings",
    when(col("playtime_forever") <= col("cut_point_1"), lit(1))
    .when((col("playtime_forever") > col("cut_point_1")) & (col("playtime_forever") <= col("cut_point_2")), lit(2))
    .when((col("playtime_forever") > col("cut_point_2")) & (col("playtime_forever") <= col("cut_point_3")), lit(3))
    .when((col("playtime_forever") > col("cut_point_3")) & (col("playtime_forever") <= col("cut_point_4")), lit(4))
    .otherwise(lit(5))
)

training.show()

+------+-----------------+----------------+------------------+--------------------+-----------------+-----------+-----------------+------------------+-------+
| appid|          steamid|playtime_forever|game_mean_playtime|game_stddev_playtime|      cut_point_1|cut_point_2|      cut_point_3|       cut_point_4|ratings|
+------+-----------------+----------------+------------------+--------------------+-----------------+-----------+-----------------+------------------+-------+
|263700|76561198071863000|              34|          109.0000|                75.0|             71.5|   109.0000|            146.5|             184.0|      1|
|263700|76561197968595000|             184|          109.0000|                75.0|             71.5|   109.0000|            146.5|             184.0|      4|
| 49900|76561198062015000|              23|           99.9355|  138.69656727242017|30.58721636378992|    99.9355|169.2837836362101|238.63206727242016|      1|
| 49900|76561198049923000|              32|   

## Pearson Correlation Matrix
Since the dataset contains at most ~4500 games, we can expect a 4500^2=81,000,000 sized matrix. Each float entry takes 4 bytes in memory. Therefore, the pearson correlation matrix would take up 324 MB of memory.

Since the memory used is relatively small, we will pre-compute the person correlation matrix and store it for use later in the algorithm.

To start off, we create a list of features for each game (appid) using the playtime_forever of all users who have played the game.

In [6]:
# Compute the maximum length of the lists of ratings values
max_len = training.filter("ratings IS NOT NULL") \
    .groupBy('appid').agg(size(collect_list('ratings')).alias('num_playtimes')) \
    .agg({'num_playtimes': 'max'}).collect()[0][0]

# Define a UDF to pad lists with zeros
pad_zeros = udf(lambda x: x + [0.0]*(max_len-len(x)), ArrayType(DoubleType()))

# Create playtime vectors for each game
list_to_dense = udf(lambda l: Vectors.dense(l), VectorUDT())
vectors = training.filter("ratings IS NOT NULL")\
    .groupBy('appid').agg(collect_list('ratings'))\
        .withColumn('padded_features', pad_zeros('collect_list(ratings)')) \
        .withColumn('features', list_to_dense('padded_features'))

We add a row number column to the dataframe so that we can match the appid with the row number in the correlation matrix.

In [7]:
# Add a row number column to the game matrix
windowSpec = Window.orderBy("appid")
vectors = vectors.withColumn("row_num", row_number().over(windowSpec))
vectors.show()

# Create a dictionary of appids and row numbers
all_row_num = vectors.select("appid", "row_num").rdd.collectAsMap()

+-----+---------------------+--------------------+--------------------+-------+
|appid|collect_list(ratings)|     padded_features|            features|row_num|
+-----+---------------------+--------------------+--------------------+-------+
|  100| [2, 3, 2, 2, 2, 2...|[null, null, null...|[2.0,3.0,2.0,2.0,...|      1|
|  300| [2, 2, 2, 2, 3, 2...|[null, null, null...|[2.0,2.0,2.0,2.0,...|      2|
|  400| [2, 2, 2, 2, 2, 2...|[null, null, null...|[2.0,2.0,2.0,2.0,...|      3|
|  500| [2, 2, 3, 2, 2, 2...|[null, null, null...|[2.0,2.0,3.0,2.0,...|      4|
| 1200| [2, 2, 2, 2, 2, 2...|[null, null, null...|[2.0,2.0,2.0,2.0,...|      5|
| 1300| [2, 1, 2, 1, 1, 5...|[null, null, null...|[2.0,1.0,2.0,1.0,...|      6|
| 1500| [2, 2, 2, 2, 2, 2...|[null, null, null...|[2.0,2.0,2.0,2.0,...|      7|
| 1600| [1, 5, 1, 5, 3, 3...|[null, null, null...|[1.0,5.0,1.0,5.0,...|      8|
| 1700| [5, 5, 1, 5, 1, 2...|[null, null, null...|[5.0,5.0,1.0,5.0,...|      9|
| 1900| [2, 3, 2, 3, 3, 2...|[null, null

Using the correlation method provided by Pyspark, we compute the correlation matrix.

In [8]:
pearson_matrix = Correlation.corr(vectors.orderBy("row_num"), "features", "pearson")
corr_array = pearson_matrix.head()[0].toArray()
print(corr_array)

[[ 1.         -0.00671087  0.08301802 ... -0.00418148 -0.00418148
  -0.00418148]
 [-0.00671087  1.          0.08396612 ... -0.00362308 -0.00362308
  -0.00362308]
 [ 0.08301802  0.08396612  1.         ...  0.00190252  0.00190252
   0.00190252]
 ...
 [-0.00418148 -0.00362308  0.00190252 ...  1.          1.
   1.        ]
 [-0.00418148 -0.00362308  0.00190252 ...  1.          1.
   1.        ]
 [-0.00418148 -0.00362308  0.00190252 ...  1.          1.
   1.        ]]


## Rating Prediction


For a given user (steamid) and game (appid), we can compute the predicted rating of the game (appid) for the user (steamid) using the weighted average of the k most similar games to the game (appid) that the user (steamid) has played.

This is the same item-item collaborative filtering formula that was shown in class.

In [24]:
def predict_rating(appid, user_ratings_dict):

    # Get appid row number from the vectors dataframe
    appid_row_num = all_row_num[appid]

    # Get a list of correlations between the appid and all other games
    corr = corr_array[appid_row_num]

    # Set the correlation to 0 for appids that the user has not rated
    for appid in all_row_num:
        if appid not in user_ratings_dict:
            corr[all_row_num[appid]] = 0

    # Make a list of tuples of (appid, correlation, rating)
    corr_list = []
    for appid in all_row_num:
        if appid in user_ratings_dict:
            row_num = all_row_num[appid]
            corr_list.append((appid, corr[row_num], user_ratings_dict[appid]))

    # Sort the list by correlation
    corr_list.sort(key=lambda x: x[1], reverse=True)

    # Get the top 10 most similar appids
    top_10 = corr_list[1:11]

    # Calculate the weighted average of the top 10 appids
    numerator = 0
    denominator = 0
    for appid, corr, rating in top_10:
        numerator += corr * rating
        denominator += corr
    if denominator != 0:
        return numerator / denominator
    else:
        return 0

def predict_single_rating(steamid, appid):
    # Get all the user's ratings
    user_ratings_dict = training.filter(col("steamid") == steamid).select("appid", "ratings").rdd.collectAsMap()
    return predict_rating(appid, user_ratings_dict)

In [25]:
# Test the function for a given user and game
test_steamid = 76561198023872000
test_appid = 500
print(f'Predicted rating for steamid [{test_steamid}] and appid [{test_appid}]: {predict_single_rating(test_steamid, test_appid)}')

Predicted rating for steamid [76561198023872000] and appid [500]: 2.8795349009874287


## Recommender



In [26]:
# Get all unique appids
all_appids = vectors.select("appid").rdd.flatMap(lambda x: x).collect()

624


In [27]:
def predict_user_ratings(steamid, recommendation_count=3):
    # Get all_appids that the user has not rated
    user_ratings_dict = training.filter(col("steamid") == steamid).select("appid", "ratings").rdd.collectAsMap()
    not_rated = [appid for appid in all_appids if appid not in user_ratings_dict]
    not_rated_rdd = spark.sparkContext.parallelize(not_rated)
    # Run predict_rating for each appid and output a list of tuples of (appid, predicted rating)
    predictions = not_rated_rdd.map(lambda appid: (appid, predict_rating(appid, user_ratings_dict))).collect()
    # predictions = [(appid, predict_rating(steamid, appid)) for appid in not_rated]
    # Sort the list by predicted rating
    predictions.sort(key=lambda x: x[1], reverse=True)
    # Return top 3 appids outside of the tuple
    return [appid for appid, rating in predictions[:recommendation_count]]

print(f'Predicted top 3 games for steamid [{test_steamid}]: {predict_user_ratings(test_steamid)}')

Predicted top 3 games for steamid [76561198023872000]: [Decimal('45300'), Decimal('47500'), Decimal('43500')]
